In [1]:
from matplotlib import pyplot as plt
import numpy as np
from numpy import linalg as la
import matplotlib.animation as animation
from IPython.display import HTML

In [2]:
class TestFunction:
    def __init__(self, name: str, domain: np.ndarray, glob_min: np.ndarray, fun: callable) -> None:

        """_summary_

        Args:
            domain (np.ndarray): Область определения функции
            glob_min (np.ndarray): Глобальный минимум
            fun (callable): Функция
        """
        self.name = name
        self.domain = domain
        self.glob_min = glob_min
        self.fun = fun

In [3]:
Sphere = TestFunction(
    name = "Sphere",
    fun = lambda p: p[0]*p[0] + p[1]*p[1],
    domain = np.array([[-3., -3.], [3., 3.]]),
    glob_min = np.array([0., 0., 0.])
    )

In [4]:
McCormick = TestFunction(
    name = "McCormick",
    fun = lambda p: np.sin(p[0] + p[1]) + (p[0] - p[1]) ** 2 - 1.5 * p[0] + 2.5 * p[1] + 1,
    domain = np.array([[-1.5, -3.], [5., 4.]]),
    glob_min = np.array([-0.54719, -1.54719, -1.9133])
    )

In [43]:
Booth = TestFunction(
    name = "Booth",
    fun = lambda p: (p[0] + 2 * p[1] - 7) ** 2 + (2 * p[0] + p[1] - 5) ** 2,
    domain = np.array([[-3., -3.], [7., 7.]]),
    glob_min = np.array([1., 3., 0.])
)

In [5]:
def grad(fun: callable, point: np.ndarray, dt: float = 0.00001) -> np.array:

    """Вычисляет градиент функции в точке

    Args:
        fun (callable): Функция искуственного ландшавта
        point (np.ndarray): Точка (массив параметров)
        dt (float, optional): Изменение аргумента. Defaults to 0.00001.

    Returns:
        np.array: _description_
    """

    dxdt = (fun(point + np.array([dt, 0])) - fun(point)) / dt
    dydt = (fun(point + np.array([0, dt])) - fun(point)) / dt
    return np.array([dxdt, dydt])

In [6]:
def classic_GD(
    fun: callable, start_params: np.ndarray, glob_min: np.ndarray,
    max_iter: int = 64, lr: float = 0.1, delta: float = 0.001
    ) -> np.array:

    """Классический градиентный спуск

    Args:
        fun (callable): Функция искуственного ландшавта
        start_params (np.ndarray): Начальный набор параметров
        glob_min (np.ndarray): Глобальный минимум рассматриваемой функции
        max_iter (int, optional): Ограничение по кол-ву итераций. Defaults to 64.
        lr (float, optional): Шаг обучения. Defaults to 0.1.
        delta (float, optional): Радиус сходимости. Defaults to 0.001.

    Returns:
        np.array: История градиентного спуска
    """

    # Рассчитываем начальный набор параметров
    params = start_params.copy()
    path = [np.array([params[0], params[1], fun(params)])]

    step = 0
    while (step < max_iter and la.norm(path[-1] - glob_min) > delta):

        # Вычисляем новое значение параметров
        params = params - lr * grad(fun, params)

        # Логируем результат
        path.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(path)


In [33]:
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.animation as animation

draw_legend = True

def update_lines(num, ax):
    global draw_legend
    line = ax.plot(path[:num, 0], path[:num, 1], path[:num, 2], '-o', c='black', label = 'Градиентный спуск', alpha = 0.7)
    if draw_legend:
        draw_legend = False
        ax.legend(loc="upper left")

    return line

def draw_result(fun: callable, domain: np.array, glob_min: np.array, path: np.array, title: str) -> None:

    """Визуализация градиентного спуска

    Args:
        fun (callable): Исходная функция
        domain (np.array): Границы поиска
        glob_min (np.array): Глобальный минимум функции
        path (np.array): История градиентного спуска
        title (str): Содержание заголовка
    """

    global draw_legend
    draw_legend = True

    fig = plt.figure(figsize = (10, 10))
    ax = plt.axes(projection = '3d')

    print(domain)

    x = np.linspace(domain[0, 0], domain[1, 0], 100)
    y = np.linspace(domain[0, 1], domain[1, 1], 100)

    x_grid, y_grid = np.meshgrid(x, y)
    z_grid = fun(np.array([x_grid, y_grid]))

    ax.plot_surface(x_grid, y_grid, z_grid, cmap = 'plasma', alpha=0.5)
    ax.scatter3D(path[0, 0], path[0, 1], path[0, 2], s=100, c="white", lw=2, ec='black', marker = 'D', label="Начальная точка")
    ax.scatter3D(path[-1, 0], path[-1, 1], path[-1, 2], s=190, c="white", lw=2, ec='black', marker = 'X', label="Найденный минимум")
    ax.scatter3D(glob_min[0], glob_min[1], glob_min[2], s=150, c="red", lw=2, ec='black', marker = 'o', label="Глобальный минимум", alpha = 0.7)

    np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
    print(f"Начальная точка:\t\t\t{path[0]}")
    np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
    print(f"Найденный минимум:\t\t\t{path[-1]}")
    print(f"Глобальный минимум:\t\t\t{glob_min}")
    print(f"Кол-во итераций:\t\t\t{len(path)}")
    np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
    print(f"Погрешность найденного решения:\t\t{glob_min - path[-1]}")
    fig.text(0.9, 0.1, s=f"Кол-во итераций: {len(path)}", horizontalalignment="right", fontsize = 12)

    ax.set_title(title, fontsize = 12, fontweight="bold",loc="left")
    ax.legend(loc="upper left")
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    num_steps = min(100, len(path))
    ani = animation.FuncAnimation(
            fig, update_lines, num_steps, fargs=(ax,), interval=50)

    return ani

plt.ioff()

In [18]:
path = classic_GD(Sphere.fun, np.array([2.0, 2.7]), Sphere.glob_min, delta = 0.00001)
anim = draw_result(Sphere.fun, Sphere.domain, Sphere.glob_min, path, "Классический градиентный спуск")
plt.ioff()

Начальная точка:			[2.00 2.70 11.29]
Найденный минимум:			[4.35e-06 7.63e-06 7.71e-11]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			56
Погрешность найденного решения:		[-4.35e-06 -7.63e-06 -7.71e-11]


In [19]:
HTML(anim.to_html5_video())

In [20]:
path = classic_GD(McCormick.fun, np.array([-1.0, 3.0]), McCormick.glob_min)
anim = draw_result(McCormick.fun, McCormick.domain, McCormick.glob_min, path, "Классический градиентный спуск")
plt.ioff()

Начальная точка:			[-1.00 3.00 26.91]
Найденный минимум:			[-5.47e-01 -1.55e+00 -1.91e+00]
Глобальный минимум:			[-5.47e-01 -1.55e+00 -1.91e+00]
Кол-во итераций:			57
Погрешность найденного решения:		[-6.60e-04 -6.60e-04 -7.78e-05]


In [21]:
HTML(anim.to_html5_video())

In [45]:
path = classic_GD(Booth.fun, np.array([5., 0.]), Booth.glob_min, delta = 0.00001)
anim = draw_result(Booth.fun, Booth.domain, Booth.glob_min, path, "Классический градиентный спуск")
plt.ioff()

[[-3.00e+00 -3.00e+00]
 [7.00e+00 7.00e+00]]
Начальная точка:			[5.00 0.00 29.00]
Найденный минимум:			[1.00e+00 3.00e+00 1.30e-10]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			61
Погрешность найденного решения:		[-3.35e-06 7.38e-06 -1.30e-10]


In [46]:
HTML(anim.to_html5_video())